##======================== 0. Set library

In [2]:

library(data.table)
library(tidyverse)
library(tidyr)
library(plyr)
library(reshape2)
library(ggpubr)
library(cowplot) # use merged plot


── Attaching packages ─────────────────────────────────────── tidyverse 1.2.1 ──
✔ ggplot2 3.0.0     ✔ purrr   0.2.5
✔ tibble  1.4.2     ✔ dplyr   0.7.6
✔ tidyr   0.8.1     ✔ stringr 1.3.1
✔ readr   1.1.1     ✔ forcats 0.3.0
── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::between()   masks data.table::between()
✖ dplyr::filter()    masks stats::filter()
✖ dplyr::first()     masks data.table::first()
✖ dplyr::lag()       masks stats::lag()
✖ dplyr::last()      masks data.table::last()
✖ purrr::transpose() masks data.table::transpose()
------------------------------------------------------------------------------
You have loaded plyr after dplyr - this is likely to cause problems.
If you need functions from both plyr and dplyr, please load plyr first, then dplyr:
library(plyr); library(dplyr)
------------------------------------------------------------------------------

Attaching package: ‘plyr’

The following objects are masked from ‘package:dp

# 0. data

In [3]:

# input files
f_drugList <- "~/Documents/2.proj-PDX/2.analysis_pdxEvolution/analysis-therapeuticEvent/v2/_nci-match/nci-match_eay131.20200821.formatted.table"
f_somaticMut <- "~/Documents/2.proj-PDX/2.analysis_pdxEvolution/v5.datafreeze/2.somaticMut/datafreeze.somaticMut.all.non-silent.meta3.formal.v5.20200427.tsv"
f_cnv <- "~/Documents/2.proj-PDX/2.analysis_pdxEvolution/v5.datafreeze/3.cnv/dataFreezeSamples.cnvkit.geneLevel.from_segment.v5.tsv"
f_geneExp <- "~/Documents/2.proj-PDX/2.analysis_pdxEvolution/v5.datafreeze/5.geneExp/pdmr_other_washU-b1-b8.datafreeze.v5.kallisto.tsv"
f_fusion <- "~/Documents/2.proj-PDX/2.analysis_pdxEvolution/v5.datafreeze/6.fusion/star-fusion.merged.FusionInspector.fusions.abridged.annotated.coding_effect.filtered.v5.tsv" 
#f_sampleInfo <- "~/Documents/2.proj-PDX/2.analysis_pdxEvolution/v5.datafreeze/1.sampleInfo/sampleInfo.washU_b1-b8.pdmr.other.passed.v5.extra.20200401.v2.tsv" 
f_sampleInfo <- "~/Documents/2.proj-PDX/2.analysis_pdxEvolution/0.sampleInfo/4.merged/v5/sampleInfo.washU_b1-b8.pdmr.other.passed.v5.extra.20200401.v4.1.tsv" 
f_info_wxs_matched_rna <- "~/Documents/2.proj-PDX/2.analysis_pdxEvolution/0.sampleInfo/4.merged/v5/pair_table/all.WES_with_RNASEQ.HT_PDX.sampleinfo.tsv"

# read files
d_drugSet <- fread(f_drugList, data.table=F)
d_geneExp <- fread(f_geneExp, data.table=F)
d_sampleInfo <- fread(f_sampleInfo, data.table=F)
d_info_wxs_rna <- fread(f_info_wxs_matched_rna, data.table=F)

# make PDX only data_info that WES with RNA-Seq
d_pdx_info <- d_info_wxs_rna[d_info_wxs_rna$Group=="PDX",]
    # Source CancerGroup CancerType CaseID  SampleID Group Passage RNASEQ  WES


# output analysis results
outdir <- "."
if (! dir.exists(outdir)) { dir.create(outdir) }


# Set functions

In [4]:
#--------- (Important checking) make text file for record siginficant amplification

##======= set functions for checking significantly diff mut in exp

func_rec_sigDiffExp_in_alt <- function(df, alteration){
    # df
    # CancerType, SampleID, gene, altGroup, log2tpm
    
    d_rec <- setNames(data.frame(matrix(ncol = 7, nrow = 0)), c("CancerType", "Gene", "Alt_event", "n_total", "n_Ref", "n_Alt", "wilcoxon.p-val"))
    alt_event <- alteration
    
    for(cancerType in unique(df$CancerType)){
    
        df_cancer <- df[df$CancerType==cancerType,]
        # filter sample size < 10
        if(length(unique(df_cancer$SampleID))<10){ next }
        
        for(i in unique(df$gene)){
            
            df2 <- df_cancer[df_cancer$gene==i,]
            
            # remove genes 
            # if no any mut then del. (for pdx 1 sample also possible)
            if(alteration == 'Fusion'){
                n_total <- length(unique(df2$RNASEQ))
                n_alt <- length(unique(df2$RNASEQ[df2$altGroup==alteration]))
            } else {
                n_total <- length(unique(df2$WES))
                n_alt <- length(unique(df2$WES[df2$altGroup==alteration]))
            }
            
            if(n_alt < 3){ next }
            
            # set altGroup must 2
            if(length(unique(df2$altGroup))<2){next}
            
            # 2.not significant in gene exp. between two groups
            w_test <- wilcox.test(log2tpm ~ altGroup, data = df2)
            if(w_test$p.value >= 0.05){ next }

            # output significant
            if (w_test$p.value < 0.05){
                p_val <- formatC(w_test$p.value, format = "e", digits = 1)
               # fdr <- p.adjust(p_val, method = "fdr", n = length(p_val))
                n_ref <- n_total - n_alt
                str <- c(cancerType, i, alt_event, n_total, n_ref, n_alt, p_val)
                d_rec <- rbindlist(list(d_rec, as.list(str)))
            }
        }
    }

    # output
    if (is.data.frame(d_rec) && nrow(d_rec)==0) { return('empty')}
    d_rec <- d_rec[order(Gene, CancerType),]
    
    
    return(d_rec)
}





# 1. Somatic mutations and gene expression

In [5]:

# outdir
outdir_sub <- "../1.somaticMut_with_geneExp"
if (! dir.exists(outdir_sub)) { dir.create(outdir_sub) }


#---------- a) extract drug target
mut_tarGeneSet <- d_drugSet[d_drugSet$Event=="Mutation",]$Gene


#---------- b) extract mutation
d_somaticMut <- fread(f_somaticMut, data.table=F)

d_somaticMut_tar <- d_somaticMut %>%
                        filter(Tumor_Sample_Barcode %in% d_pdx_info$WES) %>%
                        filter(Hugo_Symbol %in% mut_tarGeneSet)

d_somaticMut_tar <- d_somaticMut_tar[,c('Tumor_Sample_Barcode', 'Hugo_Symbol', 'HGVSp_Short')]
rownames(d_somaticMut_tar) <- NULL

# merge to one gene if the gene has multiple mutations
# gene mut1,mut2,...
d_somaticMut_tar <- ddply(d_somaticMut_tar, .(Tumor_Sample_Barcode, Hugo_Symbol), summarize,
                        HGVSp_Short=paste(HGVSp_Short, collapse=",")
                    )
colnames(d_somaticMut_tar) <- c('WES', 'gene', 'mut')
# WES gene mut


# output text file for checking
write.table(d_somaticMut_tar, paste0(outdir_sub,'/1.all.target.somaticMut.',Sys.Date(),'.tsv'), sep = "\t", row.names=FALSE, quote = FALSE)






#---------- c) extract gene expression

# format gene exp data frame as --> sample gene log2tpm
d_geneExp_tar <- subset(d_geneExp, Name %in% mut_tarGeneSet)
d_geneExp_tar <- reshape2::melt(d_geneExp_tar, id.vars=c("Name"))
colnames(d_geneExp_tar) <- c('gene', 'RNASEQ', 'tpm')
d_geneExp_tar <- d_geneExp_tar[, c('RNASEQ', 'gene', 'tpm')]
# RNASEQ gene tpm

# tpm --> log2(tpm+1)
d_geneExp_tar$log2tpm <- log2(d_geneExp_tar$tpm + 1)

d_geneExp_tar <- d_geneExp_tar %>% filter(RNASEQ %in% d_pdx_info$RNASEQ)
rownames(d_geneExp_tar) <- NULL
# RNASEQ gene tpm log2tpm


#---------- d) merge formatted gene exp with somatic mut datafrme

#NOTE- must mut data merge to gene exp rather than gene exp merge to mut

#1. merge data info
d_tar_expMut <- merge(d_geneExp_tar, d_pdx_info, on='RNASEQ', all.x=TRUE)
#2. merge mut
d_tar_expMut <- merge(d_tar_expMut, d_somaticMut_tar, on=c('WES', 'gene'), all.x=TRUE)

#3. add PDXmodelID
d_tar_expMut$PDXmodelID <- d_sampleInfo$ModelID_proper[match(d_tar_expMut$WES, d_sampleInfo$Analysis_ID)]



#---------- e) add flag with Mut & Non-mut
d_tar_expMut$altGroup <- 'WT'
d_tar_expMut$altGroup[d_tar_expMut$mut != 'NA'] <- 'Mutation'


# output text file for checking
write.table(d_tar_expMut, paste0(outdir_sub,'/2.all.target.geneExp_with_somaticMut.',Sys.Date(),'.tsv'), sep = "\t", row.names=FALSE, quote = FALSE)

head(d_tar_expMut)



gene,WES,RNASEQ,tpm,log2tpm,Source,CancerGroup,CancerType,CaseID,SampleID,Group,Passage,mut,PDXmodelID,altGroup
AKT1,10960X12_HCI-009_PDXtumor_P3,10962X12_HCI009_PDXtumor_P3,120.23551,6.921669,HCI,Breast,BRCA,HCI-HCI009,HCI-HCI009.P3,PDX,P3,NA,HCI009,WT
AKT1,10960X13_HCI-009_PDXtumor_P6,10962X13_HCI009_PDXtumor_P6,110.21338,6.797187,HCI,Breast,BRCA,HCI-HCI009,HCI-HCI009.P6,PDX,P6,NA,HCI009,WT
AKT1,10960X14_HCI-010_PDXtumor_P3,10962X14_HCI010_PDXtumor_P3,33.39358,5.104067,HCI,Breast,BRCA,HCI-HCI010,HCI-HCI010.P3,PDX,P3,NA,HCI010,WT
AKT1,10960X15_HCI011_PDXtumor_P3,10962X15_HCI011_PDXtumor_P3,151.11013,7.248972,HCI,Breast,BRCA,HCI-HCI011,HCI-HCI011.P3,PDX,P3,NA,HCI011,WT
AKT1,10960X16_HCI012_PDXtumor_P4,10962X16_HCI012_PDXtumor_P4,184.41844,7.534641,HCI,Breast,BRCA,HCI-HCI012,HCI-HCI012.P4,PDX,P4,NA,HCI012,WT
AKT1,10960X17_HCI013_PDXtumor_P3,10962X17_HCI013_PDXtumor_P3,19.79917,4.378454,HCI,Breast,BRCA,HCI-HCI013,HCI-HCI013.P3,PDX,P3,NA,HCI013,WT


In [6]:

#---------- f) make visulization (1.set plot function)

library(ggplot2)
library(ggpubr)


## two group with statistic
CompBoxplot2exp <- function(title, dataFrame){
    
    #max.y <- max(dataFrame$log2tpm)
    #n_mut_case <- length(unique(dataFrame$CaseID[dataFrame$altGroup=='Mutation']))
    #n_other_case <- length(unique(dataFrame$CaseID[dataFrame$altGroup=='WT']))
    n_mut_case <- length(unique(dataFrame$PDXmodelID[dataFrame$altGroup=='Mutation']))
    n_other_case <- length(unique(dataFrame$PDXmodelID[dataFrame$altGroup=='WT']))
    
    n_mut_sample <- length(unique(dataFrame$WES[dataFrame$altGroup=='Mutation']))
    n_other_sample <- length(unique(dataFrame$WES[dataFrame$altGroup=='WT']))
    
    label_mut <- paste0("Mutation","\nN=",n_mut_case,"\nn=",n_mut_sample)
    label_other <- paste0("WT","\nN=",n_other_case,"\nn=",n_other_sample)
    

    
#    n_mut <- nrow(dataFrame[dataFrame$altGroup=='Mutation',])
#    n_other <- nrow(dataFrame[dataFrame$altGroup=='Other',])
    
    p <- ggplot(dataFrame, aes(x=altGroup, y=log2tpm, color=altGroup)) +
            geom_boxplot(lwd=0.6, outlier.size=0.6) + 
            geom_jitter(position=position_jitter(0.1), size=0.6, alpha = 0.5) +
            theme_classic() +
            scale_color_manual(values = c("WT" = "#B5B5B5", "Mutation" = "#4EAF49")) + 
            theme(legend.position="None") +
            ggtitle(title) +
            xlab("") +
            ylab("Gene expression\nlog2(TPM+1)") + 
            scale_x_discrete(
                limit = c("WT", "Mutation"),
                labels=c("WT" = label_other, "Mutation" = label_mut)
            ) +
            theme(
                plot.title = element_text(size=7, hjust = 0.5),
                axis.text = element_text(size = 6),
                axis.title=element_text(size=6)) + 
            theme(
                axis.ticks = element_line(size = rel(0.5)),
                axis.ticks.length=unit(1.5, "mm"),
                axis.line = element_line(colour = 'black', size = 0.2)
            ) +
            theme(panel.grid.major = element_blank(), 
               panel.grid.minor = element_blank(), 
               panel.background = element_blank(), 
               panel.border = element_blank()
            )
    
    #p2 <- p + stat_compare_means(size = 1, method = "wilcox.test", label.x = 1, label.y = max.y*0.9)
    
    
    
    return(p)      
}





#---------- f) make visulization (2.plotting)


for(cancerType in unique(d_tar_expMut$CancerType)){
    
    df_cancer <- d_tar_expMut[d_tar_expMut$CancerType==cancerType,]
    # filter sample size < 10
    if(length(unique(df_cancer$SampleID))<10){ next }
    
    
    pdf(paste0(outdir_sub,'/',cancerType,'.mut_geneExp.',Sys.Date(),'.pdf'), width = 1.4, height = 1.8, useDingbats=FALSE)

    for(i in unique(d_tar_expMut$gene)){
    
        df2 <- df_cancer[df_cancer$gene==i,]
        mutationSite <- unique(df2$mut[df2$altGroup!="WT"])
        #print(i)
    
        # remove genes 
        # if no any mut then del. (for pdx 1 sample also possible)
        if(nrow(df2[df2$altGroup=='Mutation',])==0){ 
            #print(paste0("[Warning] No mutation in the gene ", i, " !"))
            next
        }
        
        # set altGroup must 2
        if(length(unique(df2$altGroup))<2){next}
        
        # 2.not significant in gene exp. between two groups
        w_test <- wilcox.test(log2tpm ~ altGroup, data = df2)
        if(w_test$p.value >= 0.05){
            #print(paste0("[Warning] Not significant in the gene ", i, " ", w_test$p.value))
            next
        }
    
    
        # output significant
        if (nrow(df2[df2$altGroup=='Mutation',])>0 && w_test$p.value < 0.05){
            
            p_val <- formatC(w_test$p.value, format = "e", digits = 1)
           # q_val <- p.adjust(p_val, method = "fdr", n = length(p_val))
            title <- paste0(i,':',mutationSite , ' (', cancerType, ')', "\nWilcoxon, p=", p_val)
            
            #title <- paste0(i, ' (', cancerType, ')')
            #p_val <- paste0('Wilcoxon, p=', format(w_test$p.value, scientific = TRUE))
            p1 <- CompBoxplot2exp(title, df2)
            #p_passage <- CompBoxplot2exp_passage(title, df2)
    
            # merged plot
            #p_merged <- plot_grid(p1, p_passage, nrow=1, align="h", rel_widths = c(1, 2), labels = c('A', 'B'))
    
            #print(p_merged)
            print(p1)
        }
    
    }

    dev.off()

}



In [7]:
#--------- (Important checking) make text file for record siginficant amplification
alterationEvent <- 'Mutation'

head(d_tar_expMut)

d_rec <- func_rec_sigDiffExp_in_alt(d_tar_expMut, alterationEvent)

write.table(d_rec, paste0(outdir_sub,'/1.p-val.target.geneExp_with_',alterationEvent,'.',Sys.Date(),'.tsv'), sep = "\t", row.names=FALSE, quote = FALSE)



gene,WES,RNASEQ,tpm,log2tpm,Source,CancerGroup,CancerType,CaseID,SampleID,Group,Passage,mut,PDXmodelID,altGroup
AKT1,10960X12_HCI-009_PDXtumor_P3,10962X12_HCI009_PDXtumor_P3,120.23551,6.921669,HCI,Breast,BRCA,HCI-HCI009,HCI-HCI009.P3,PDX,P3,NA,HCI009,WT
AKT1,10960X13_HCI-009_PDXtumor_P6,10962X13_HCI009_PDXtumor_P6,110.21338,6.797187,HCI,Breast,BRCA,HCI-HCI009,HCI-HCI009.P6,PDX,P6,NA,HCI009,WT
AKT1,10960X14_HCI-010_PDXtumor_P3,10962X14_HCI010_PDXtumor_P3,33.39358,5.104067,HCI,Breast,BRCA,HCI-HCI010,HCI-HCI010.P3,PDX,P3,NA,HCI010,WT
AKT1,10960X15_HCI011_PDXtumor_P3,10962X15_HCI011_PDXtumor_P3,151.11013,7.248972,HCI,Breast,BRCA,HCI-HCI011,HCI-HCI011.P3,PDX,P3,NA,HCI011,WT
AKT1,10960X16_HCI012_PDXtumor_P4,10962X16_HCI012_PDXtumor_P4,184.41844,7.534641,HCI,Breast,BRCA,HCI-HCI012,HCI-HCI012.P4,PDX,P4,NA,HCI012,WT
AKT1,10960X17_HCI013_PDXtumor_P3,10962X17_HCI013_PDXtumor_P3,19.79917,4.378454,HCI,Breast,BRCA,HCI-HCI013,HCI-HCI013.P3,PDX,P3,NA,HCI013,WT


# 2. Fusions and gene expression

In [9]:
d_fusion <- fread(f_fusion, data.table=F)

# outdir
outdir_sub <- "../2.fusion_with_geneExp"
if (! dir.exists(outdir_sub)) { dir.create(outdir_sub) }


#---------- prepare info
d_sampleInfo_pdx_rna <- d_sampleInfo[d_sampleInfo$Group=="PDX" & d_sampleInfo$DataType=="RNA-Seq",]
d_sampleInfo_pdx_rna <- d_sampleInfo_pdx_rna[,c('Analysis_ID', 'Source', 'CancerGroup', 'CancerType', 'Formal_CaseID', 'NewSampleID', 'Group', 'NCI_Passage', 'ModelID_proper_v2')]


#---------- a) extract drug target
fusion_tarGeneSet <- d_drugSet[d_drugSet$Event=="Fusion",]$Gene


#---------- b) extract fusion for pdx
d_fusion <- fread(f_fusion, data.table=F)
d_fusion_pdx <- d_fusion[d_fusion$Sample %in% d_sampleInfo_pdx_rna$Analysis_ID,]


#---------- c) format fusion data for extraction

## set function for fomatting
func_extractTargetFusion <- function(geneList, df_starFusion)
{
    # reset rownames
    rownames(df_starFusion) <- NULL
    max_row = nrow(df_starFusion)
    
    # new dataframe
    d_matchedFusion <- setNames(data.frame(matrix(ncol = 4, nrow = 0)), c("RNASEQ", "gene", "fusionName", "fusionType"))
    
    
    for (i in 1:max_row){
        
        fusionName = df_starFusion[i,]$FusionName
        
        for (gene in geneList){    

            if (length(grep(paste0("^",gene,"--"), fusionName))>0 || length(grep(paste0("--",gene,"$"), fusionName))>0){
                RNASEQ = df_starFusion[i,]$Sample
                fusionName = df_starFusion[i,]$FusionName
                fusionType = df_starFusion[i,]$PROT_FUSION_TYPE
                
                fusion_info <- c(RNASEQ, gene, fusionName, fusionType)
                
                d_matchedFusion <- rbindlist(list(d_matchedFusion, as.list(fusion_info)))
                
            } else {
                next
            }
            
        }
    }
    
    return(d_matchedFusion)
}



# extract target gene fusion of pdx samples
d_fusion_tar <- func_extractTargetFusion(fusion_tarGeneSet, d_fusion_pdx)
# "RNASEQ", "gene", "fusionName", "fusionType"


# merge to one gene if the gene has multiple fusions
library(plyr)
d_fusion_tar <- ddply(d_fusion_tar, .(RNASEQ, gene), summarize,
                    fusionName=paste(fusionName,collapse=","), 
                    fusionType=paste(fusionType,collapse=",")
                )
# "RNASEQ", "gene", "fusionName", "fusionType"


head(d_fusion_tar)


#---------- d) extract gene expression

# format gene exp data frame as --> sample gene log2tpm
d_geneExp_tar <- subset(d_geneExp, Name %in% fusion_tarGeneSet)
d_geneExp_tar <- reshape2::melt(d_geneExp_tar, id.vars=c("Name"))
colnames(d_geneExp_tar) <- c('gene', 'RNASEQ', 'tpm')
d_geneExp_tar <- d_geneExp_tar[, c('RNASEQ', 'gene', 'tpm')]
# RNASEQ gene tpm

# tpm --> log2(tpm+1)
d_geneExp_tar$log2tpm <- log2(d_geneExp_tar$tpm + 1)

d_geneExp_tar <- d_geneExp_tar %>% filter(RNASEQ %in% d_sampleInfo_pdx_rna$Analysis_ID)
rownames(d_geneExp_tar) <- NULL
# RNASEQ gene tpm log2tpm




#---------- e) merge formatted gene exp with fusion datafrme

#NOTE- must fusion data merge to gene exp rather than gene exp merge to fusion

#1. merge data info
d_tar_expFusion <- merge(d_geneExp_tar, d_sampleInfo_pdx_rna, by.x='RNASEQ', by.y="Analysis_ID", all.x=TRUE)
#2. merge fusion
d_tar_expFusion <- merge(d_tar_expFusion, d_fusion_tar, on=c('RNASEQ', 'gene'), all.x=TRUE)



#---------- f) add flag with Mut & Non-mut
d_tar_expFusion$altGroup <- 'WT'
d_tar_expFusion$altGroup[d_tar_expFusion$fusionName != 'NA'] <- 'Fusion'




# output text file for checking
write.table(d_tar_expFusion, paste0(outdir_sub,'/2.all.target.geneExp_with_fusion.',Sys.Date(),'.tsv'), sep = "\t", row.names=FALSE, quote = FALSE)

head(d_tar_expFusion)




#---------- g) make visulization (1.set plot function)

library(ggplot2)
library(ggpubr)


## two group with statistic
CompBoxplot2exp <- function(title, dataFrame){   

    #n_alt_case <- length(unique(dataFrame$Formal_CaseID[dataFrame$altGroup=='Fusion']))
    #n_other_case <- length(unique(dataFrame$Formal_CaseID[dataFrame$altGroup=='WT']))
    n_alt_case <- length(unique(dataFrame$ModelID_proper[dataFrame$altGroup=='Fusion']))
    n_other_case <- length(unique(dataFrame$ModelID_proper[dataFrame$altGroup=='WT']))
    
    n_alt_sample <- length(unique(dataFrame$RNASEQ[dataFrame$altGroup=='Fusion']))
    n_other_sample <- length(unique(dataFrame$RNASEQ[dataFrame$altGroup=='WT']))
    
    label_alt <- paste0("Fusion","\nN=",n_alt_case,"\nn=",n_alt_sample)
    label_other <- paste0("WT","\nN=",n_other_case,"\nn=",n_other_sample)
    
    
    p <- ggplot(dataFrame, aes(x=altGroup, y=log2tpm, color=altGroup)) +
            geom_boxplot(lwd=0.6, outlier.size=0.6) + 
            geom_jitter(position=position_jitter(0.1), size=0.6, alpha = 0.5) +
            theme_classic() +
            scale_color_manual(values = c("WT" = "#B5B5B5", "Fusion" = "#F57E20")) + 
            theme(legend.position="None") +
            ggtitle(title) +
            xlab("") +
            ylab("Gene expression\nlog2(TPM+1)") + 
            scale_x_discrete(
                limit = c("WT", "Fusion"),
                labels=c("WT" = label_other, "Fusion" = label_alt)) +
            theme(
                plot.title = element_text(size=7, hjust = 0.5),
                axis.text = element_text(size = 6),
                axis.title=element_text(size=6)) + 
            theme(
                axis.ticks = element_line(size = rel(0.5)),
                axis.ticks.length=unit(1.5, "mm"),
                axis.line = element_line(colour = 'black', size = 0.2)
            ) +
            theme(panel.grid.major = element_blank(), 
               panel.grid.minor = element_blank(), 
               panel.background = element_blank(), 
               panel.border = element_blank()
            )

    
    return(p)

}






#---------- g) make visulization (2.plotting)


for(cancerType in unique(d_tar_expFusion$CancerType)){
    
    #if (cancerType!="SKCM"){ next }
    
    df_cancer <- d_tar_expFusion[d_tar_expFusion$CancerType==cancerType,]
    # filter sample size < 10
    if(length(unique(df_cancer$NewSampleID))<10){ next }
    
    
    pdf(paste0(outdir_sub,'/',cancerType,'.fusion_geneExp.',Sys.Date(),'.pdf'), width = 1.4, height = 1.8, useDingbats=FALSE)

    for(i in unique(d_tar_expFusion$gene)){
    
        df2 <- df_cancer[df_cancer$gene==i,]
        
        # remove genes 
        # if no any mut then del. (for pdx 1 sample also possible)
        if(nrow(df2[df2$altGroup=='Fusion',])==0){ next }
    
        # 2.not significant in gene exp. between two groups
        w_test <- wilcox.test(log2tpm ~ altGroup, data = df2)
        if(w_test$p.value >= 0.05){ next }
    
    
        # output significant
        if (nrow(df2[df2$altGroup=='Fusion',])>0 && w_test$p.value < 0.05){
            
            p_val <- formatC(w_test$p.value, format = "e", digits = 1)
            
           # q_val <- p.adjust(p_val, method = "fdr", n = length(p_val))
            
            title <- paste0(i, ' (', cancerType, ')', "\nWilcoxon, p=", p_val)
            
            p1 <- CompBoxplot2exp(title, df2)

            print(p1)
            print(head(df2))
        }
    
    }

    dev.off()

}




RNASEQ,gene,fusionName,fusionType
114868_125_R_FCWC50_v1_2_RNASEQ,FGFR3,FGFR3--TACC3,INFRAME
114868_125_R_FCWC69_v1_2_RNASEQ,FGFR3,FGFR3--TACC3,INFRAME
114868_125_R_FEWC21_v1_2_RNASEQ,FGFR3,FGFR3--TACC3,INFRAME
114868_125_R_FEWC31_v1_2_RNASEQ,FGFR3,FGFR3--TACC3,INFRAME
182917_245_R_JWP_v2_0_1_4_0_RNASEQ,BRAF,"BRAF--POT1,BCAP29--BRAF","INFRAME,INFRAME"
182917_245_R_JXKE22_v2_0_1_4_0_RNASEQ,BRAF,"BCAP29--BRAF,BRAF--POT1","INFRAME,INFRAME"


RNASEQ,gene,tpm,log2tpm,Source,CancerGroup,CancerType,Formal_CaseID,NewSampleID,Group,NCI_Passage,ModelID_proper_v2,fusionName,fusionType,altGroup
10962X12_HCI009_PDXtumor_P3,ALK,0.000000,0.000000,HCI,Breast,BRCA,HCI-HCI009,HCI-HCI009.P3,PDX,P3,HCI009,NA,NA,WT
10962X12_HCI009_PDXtumor_P3,BRAF,9.491537,3.391154,HCI,Breast,BRCA,HCI-HCI009,HCI-HCI009.P3,PDX,P3,HCI009,NA,NA,WT
10962X12_HCI009_PDXtumor_P3,FGFR1,7.551949,3.096253,HCI,Breast,BRCA,HCI-HCI009,HCI-HCI009.P3,PDX,P3,HCI009,NA,NA,WT
10962X12_HCI009_PDXtumor_P3,FGFR2,1.076378,1.054069,HCI,Breast,BRCA,HCI-HCI009,HCI-HCI009.P3,PDX,P3,HCI009,NA,NA,WT
10962X12_HCI009_PDXtumor_P3,FGFR3,26.904380,4.802420,HCI,Breast,BRCA,HCI-HCI009,HCI-HCI009.P3,PDX,P3,HCI009,NA,NA,WT
10962X12_HCI009_PDXtumor_P3,FGFR4,126.784226,6.997566,HCI,Breast,BRCA,HCI-HCI009,HCI-HCI009.P3,PDX,P3,HCI009,NA,NA,WT


                                      RNASEQ  gene       tpm  log2tpm Source
285          114868_125_R_FCWC50_v1_2_RNASEQ FGFR3 15.524495 4.046534   PDMR
295          114868_125_R_FCWC69_v1_2_RNASEQ FGFR3 27.542306 4.835030   PDMR
305          114868_125_R_FEWC21_v1_2_RNASEQ FGFR3 17.573296 4.215158   PDMR
315          114868_125_R_FEWC31_v1_2_RNASEQ FGFR3 29.367395 4.924451   PDMR
605    137432_314_R_QVPG14_v2_0_1_4_0_RNASEQ FGFR3 23.924522 4.639494   PDMR
615 137432_314_R_QVPG20HQ2_v2_0_1_4_0_RNASEQ FGFR3  2.199631 1.677905   PDMR
    CancerGroup CancerType Formal_CaseID              NewSampleID Group
285   Head&Neck       HNSC   PDMR-114868    PDMR-114868.P1-FCWC50   PDX
295   Head&Neck       HNSC   PDMR-114868    PDMR-114868.P1-FCWC69   PDX
305   Head&Neck       HNSC   PDMR-114868    PDMR-114868.P1-FEWC21   PDX
315   Head&Neck       HNSC   PDMR-114868    PDMR-114868.P1-FEWC31   PDX
605   Head&Neck       HNSC   PDMR-137432    PDMR-137432.P1-QVPG14   PDX
615   Head&Neck       HNSC   

Warning message in wilcox.test.default(x = 2.55252266133868, y = c(0.012350926322528, :
“cannot compute exact p-value with ties”

In [10]:
#--------- (Important checking) make text file for record siginficant amplification
alterationEvent <- 'Fusion'

head(d_tar_expFusion)
names(d_tar_expFusion)[names(d_tar_expFusion) == "NewSampleID"] <- "SampleID"

d_rec <- func_rec_sigDiffExp_in_alt(d_tar_expFusion, alterationEvent)

#print(unique(d_tar_expFusion$altGroup))
write.table(d_rec, paste0(outdir_sub,'/1.p-val.target.geneExp_with_',alterationEvent,'.',Sys.Date(),'.tsv'), sep = "\t", row.names=FALSE, quote = FALSE)




RNASEQ,gene,tpm,log2tpm,Source,CancerGroup,CancerType,Formal_CaseID,NewSampleID,Group,NCI_Passage,ModelID_proper_v2,fusionName,fusionType,altGroup
10962X12_HCI009_PDXtumor_P3,ALK,0.000000,0.000000,HCI,Breast,BRCA,HCI-HCI009,HCI-HCI009.P3,PDX,P3,HCI009,NA,NA,WT
10962X12_HCI009_PDXtumor_P3,BRAF,9.491537,3.391154,HCI,Breast,BRCA,HCI-HCI009,HCI-HCI009.P3,PDX,P3,HCI009,NA,NA,WT
10962X12_HCI009_PDXtumor_P3,FGFR1,7.551949,3.096253,HCI,Breast,BRCA,HCI-HCI009,HCI-HCI009.P3,PDX,P3,HCI009,NA,NA,WT
10962X12_HCI009_PDXtumor_P3,FGFR2,1.076378,1.054069,HCI,Breast,BRCA,HCI-HCI009,HCI-HCI009.P3,PDX,P3,HCI009,NA,NA,WT
10962X12_HCI009_PDXtumor_P3,FGFR3,26.904380,4.802420,HCI,Breast,BRCA,HCI-HCI009,HCI-HCI009.P3,PDX,P3,HCI009,NA,NA,WT
10962X12_HCI009_PDXtumor_P3,FGFR4,126.784226,6.997566,HCI,Breast,BRCA,HCI-HCI009,HCI-HCI009.P3,PDX,P3,HCI009,NA,NA,WT


# 3. CNV and gene expression (a. amplification)

In [5]:

# outdir
outdir_sub <- "../3.cnv_amp_with_geneExp"
if (! dir.exists(outdir_sub)) { dir.create(outdir_sub) }


#---------- a) extract drug target
amp_tarGeneSet <- d_drugSet[d_drugSet$Event=="Amplification",]$Gene


#---------- b) extract mutation
d_cnv <- fread(f_cnv, data.table=F)

# sample gene segment_log2 segment_cn gene_log2  gene_cn
d_cnv_v2 <- d_cnv[, c('sample', 'gene', 'segment_log2', 'segment_cn')]


d_amp_tar <- d_cnv_v2 %>%
                filter(sample %in% d_pdx_info$WES) %>%
                filter(gene %in% amp_tarGeneSet) %>%
                filter(segment_cn > 5)

rownames(d_amp_tar) <- NULL


# output text file for checking
write.table(d_amp_tar, paste0(outdir_sub,'/1a.all.target.amplification.',Sys.Date(),'.tsv'), sep = "\t", row.names=FALSE, quote = FALSE)




#---------- c) extract gene expression

# format gene exp data frame as --> sample gene log2tpm
d_geneExp_tar <- subset(d_geneExp, Name %in% amp_tarGeneSet)
d_geneExp_tar <- reshape2::melt(d_geneExp_tar, id.vars=c("Name"))
colnames(d_geneExp_tar) <- c('gene', 'RNASEQ', 'tpm')
d_geneExp_tar <- d_geneExp_tar[, c('RNASEQ', 'gene', 'tpm')]
# RNASEQ gene tpm

# tpm --> log2(tpm+1)
d_geneExp_tar$log2tpm <- log2(d_geneExp_tar$tpm + 1)

d_geneExp_tar <- d_geneExp_tar %>% filter(RNASEQ %in% d_pdx_info$RNASEQ)
rownames(d_geneExp_tar) <- NULL
# RNASEQ gene tpm log2tpm


#---------- d) merge formatted gene exp with somatic mut datafrme

#NOTE- must cnv data merge to gene exp rather than gene exp merge to cnv

#1. merge data info
d_tar_expAmp <- merge(d_geneExp_tar, d_pdx_info, on='RNASEQ', all.x=TRUE, all.y=FALSE)
#2. merge cnv
d_tar_expAmp <- merge(d_tar_expAmp, d_amp_tar, by.x=c('WES', 'gene'), by.y=c('sample', 'gene'), all.x=TRUE, all.y=FALSE)


#---------- e) add flag with Mut & Non-mut
d_tar_expAmp$altGroup <- 'WT'
d_tar_expAmp$altGroup[d_tar_expAmp$segment_cn != 'NA'] <- 'Amplification'


# output text file for checking
write.table(d_tar_expAmp, paste0(outdir_sub,'/1b.all.target.geneExp_with_amplification.',Sys.Date(),'.tsv'), sep = "\t", row.names=FALSE, quote = FALSE)

head(d_tar_expAmp)



WES,gene,RNASEQ,tpm,log2tpm,Source,CancerGroup,CancerType,CaseID,SampleID,Group,Passage,segment_log2,segment_cn,altGroup
10960X12_HCI-009_PDXtumor_P3,CCND1,10962X12_HCI009_PDXtumor_P3,53.93992000,5.77978290,HCI,Breast,BRCA,HCI-HCI009,HCI-HCI009.P3,PDX,P3,NA,NA,WT
10960X12_HCI-009_PDXtumor_P3,CCND2,10962X12_HCI009_PDXtumor_P3,0.00795906,0.01143704,HCI,Breast,BRCA,HCI-HCI009,HCI-HCI009.P3,PDX,P3,NA,NA,WT
10960X12_HCI-009_PDXtumor_P3,CCND3,10962X12_HCI009_PDXtumor_P3,19.95486821,4.38921354,HCI,Breast,BRCA,HCI-HCI009,HCI-HCI009.P3,PDX,P3,NA,NA,WT
10960X12_HCI-009_PDXtumor_P3,CDK4,10962X12_HCI009_PDXtumor_P3,32.78574113,5.07834260,HCI,Breast,BRCA,HCI-HCI009,HCI-HCI009.P3,PDX,P3,NA,NA,WT
10960X12_HCI-009_PDXtumor_P3,CDK6,10962X12_HCI009_PDXtumor_P3,11.64335100,3.66030698,HCI,Breast,BRCA,HCI-HCI009,HCI-HCI009.P3,PDX,P3,NA,NA,WT
10960X12_HCI-009_PDXtumor_P3,ERBB2,10962X12_HCI009_PDXtumor_P3,75.91210600,6.26513879,HCI,Breast,BRCA,HCI-HCI009,HCI-HCI009.P3,PDX,P3,NA,NA,WT


In [6]:
#--------- (Important checking) make text file for record siginficant amplification
alterationEvent <- 'Amplification'
d_rec <- func_rec_sigDiffExp_in_alt(d_tar_expAmp, alterationEvent)
write.table(d_rec, paste0(outdir_sub,'/1.p-val.target.geneExp_with_',alterationEvent,'.',Sys.Date(),'.tsv'), sep = "\t", row.names=FALSE, quote = FALSE)



In [7]:

#---------- f) make visulization (1.set plot function)

library(ggplot2)
library(ggpubr)

CompBoxplot2exp_panCancers_per_gene <- function(gene, dataFrame){
    
    df_gene <- dataFrame[dataFrame$gene==gene,]
    
    max.y <- max(df_gene$log2tpm)
    
    p <- ggplot(df_gene, aes(x=CancerType, y=log2tpm, color=altGroup)) +
            geom_boxplot(lwd=0.4, position=position_dodge(1), outlier.size=0.4, pch=16, outlier.alpha=0.5) + 
            geom_point(pch=16, position=position_jitterdodge(0.2), size=1, alpha = 0.5) +
            theme_classic() +
            #scale_color_manual(values = c("Amplification" = "#D32A29", "WT" = "#7D94AF"), labels = c("Amplification"="Amplification", "WT"="No amplification")) +
            scale_color_manual(values = c("WT" = "#7D94AF", "Amplification" = "#D32A29")) +
            ggtitle(gene) +
            theme(plot.title = element_text(size=8,hjust = 0.5),
                    axis.title=element_text(size=7),
                    axis.text.x = element_text(size = 7, angle = 60, hjust = 1),
                    axis.text.y = element_text(size = 7)) + 
            theme(
                axis.ticks = element_line(size = rel(0.5)),
                axis.ticks.length=unit(1.5, "mm"),
                axis.line = element_line(colour = 'black', size = 0.2)) +
            xlab("") +
            ylab("Gene expression\nlog2(TPM+1)") + 
            theme(legend.position=c(0.9, 0.9),
                    legend.title=element_blank(),
                    legend.key.size = unit(0.5, "cm"),
                    legend.text=element_text(size=7)
            ) + 
            ylim(0,15)
    
    return(p)

}




#---------- f) make visulization (2.plotting)

#f_in <- paste0(outdir_sub,'/1.p-val.target.geneExp_with_Amplification','.',Sys.Date(),'.tsv')
f_in <- '../3.cnv_amp_with_geneExp/1b.all.target.geneExp_with_amplification.filteredCase.tsv'

#pdf(paste0(outdir_sub,'/','1c.all.target.geneExp_with_amplification.',Sys.Date(),'.pdf'), width = 7.5, height = 2.5, useDingbats=FALSE)

pdf(paste0(f_in,'.pdf'), width = 7.5, height = 2.5, useDingbats=FALSE)


d_tar_expAmp$altGroup <- factor(d_tar_expAmp$altGroup, levels = c('WT', 'Amplification'))

for(i in unique(d_tar_expAmp$gene)){
    
    df2 <- d_tar_expAmp[d_tar_expAmp$gene==i,]
    
    if(nrow(df2[df2$altGroup=='Amplification',])==0){
        next
    }
    
    p <- CompBoxplot2exp_panCancers_per_gene(i, df2)
    print(p)

}

dev.off()



pdf 
  2

# 3. CNV and gene expression (b. Deletion)

In [15]:
# outdir
outdir_sub <- "../3.cnv_del_with_geneExp"
if (! dir.exists(outdir_sub)) { dir.create(outdir_sub) }

altEvent <- 'Deletion'

#---------- a) extract drug target
del_tarGeneSet <- d_drugSet[d_drugSet$Event==altEvent,]$Gene


#---------- b) extract mutation
d_cnv <- fread(f_cnv, data.table=F)

# sample gene segment_log2 segment_cn gene_log2  gene_cn
d_cnv_v2 <- d_cnv[, c('sample', 'gene', 'segment_log2', 'segment_cn')]


d_del_tar <- d_cnv_v2 %>%
                filter(sample %in% d_pdx_info$WES) %>%
                filter(gene %in% amp_tarGeneSet) %>%
                filter(segment_log2 < -1.3)

rownames(d_del_tar) <- NULL


# output text file for checking
write.table(d_del_tar, paste0(outdir_sub,'/2a.all.target.',altEvent,'.',Sys.Date(),'.tsv'), sep = "\t", row.names=FALSE, quote = FALSE)




#---------- c) extract gene expression

# format gene exp data frame as --> sample gene log2tpm
d_geneExp_tar <- subset(d_geneExp, Name %in% del_tarGeneSet)
d_geneExp_tar <- reshape2::melt(d_geneExp_tar, id.vars=c("Name"))
colnames(d_geneExp_tar) <- c('gene', 'RNASEQ', 'tpm')
d_geneExp_tar <- d_geneExp_tar[, c('RNASEQ', 'gene', 'tpm')]
# RNASEQ gene tpm

# tpm --> log2(tpm+1)
d_geneExp_tar$log2tpm <- log2(d_geneExp_tar$tpm + 1)

d_geneExp_tar <- d_geneExp_tar %>% filter(RNASEQ %in% d_pdx_info$RNASEQ)
rownames(d_geneExp_tar) <- NULL
# RNASEQ gene tpm log2tpm


#---------- d) merge formatted gene exp with somatic mut datafrme

#NOTE- must cnv data merge to gene exp rather than gene exp merge to cnv

#1. merge data info
d_tar_expDel <- merge(d_geneExp_tar, d_pdx_info, on='RNASEQ', all.x=TRUE, all.y=FALSE)
#2. merge cnv
d_tar_expDel <- merge(d_tar_expDel, d_amp_tar, by.x=c('WES', 'gene'), by.y=c('sample', 'gene'), all.x=TRUE, all.y=FALSE)


#---------- e) add flag with Mut & Non-mut
d_tar_expDel$altGroup <- 'WT'
d_tar_expDel$altGroup[d_tar_expDel$segment_cn != 'NA'] <- 'Deletion'


# output text file for checking
write.table(d_tar_expDel, paste0(outdir_sub,'/2b.all.target.geneExp_with_', altEvent, '.',Sys.Date(),'.tsv'), sep = "\t", row.names=FALSE, quote = FALSE)

head(d_tar_expDel)



WES,gene,RNASEQ,tpm,log2tpm,Source,CancerGroup,CancerType,CaseID,SampleID,Group,Passage,segment_log2,segment_cn,altGroup
10960X12_HCI-009_PDXtumor_P3,MET,10962X12_HCI009_PDXtumor_P3,11.29700740,3.62023536,HCI,Breast,BRCA,HCI-HCI009,HCI-HCI009.P3,PDX,P3,NA,NA,WT
10960X12_HCI-009_PDXtumor_P3,PTEN,10962X12_HCI009_PDXtumor_P3,0.05750863,0.08066944,HCI,Breast,BRCA,HCI-HCI009,HCI-HCI009.P3,PDX,P3,NA,NA,WT
10960X13_HCI-009_PDXtumor_P6,MET,10962X13_HCI009_PDXtumor_P6,14.94561300,3.99508766,HCI,Breast,BRCA,HCI-HCI009,HCI-HCI009.P6,PDX,P6,NA,NA,WT
10960X13_HCI-009_PDXtumor_P6,PTEN,10962X13_HCI009_PDXtumor_P6,0.12914471,0.17523039,HCI,Breast,BRCA,HCI-HCI009,HCI-HCI009.P6,PDX,P6,NA,NA,WT
10960X14_HCI-010_PDXtumor_P3,MET,10962X14_HCI010_PDXtumor_P3,30.32423900,4.96920756,HCI,Breast,BRCA,HCI-HCI010,HCI-HCI010.P3,PDX,P3,NA,NA,WT
10960X14_HCI-010_PDXtumor_P3,PTEN,10962X14_HCI010_PDXtumor_P3,0.58265600,0.66234771,HCI,Breast,BRCA,HCI-HCI010,HCI-HCI010.P3,PDX,P3,NA,NA,WT


In [16]:
#--------- (Important checking) make text file for record siginficant deletion
alterationEvent <- 'Deletion'
d_rec <- func_rec_sigDiffExp_in_alt(d_tar_expDel, alterationEvent)
write.table(d_rec, paste0(outdir_sub,'/2.p-val.target.geneExp_with_',alterationEvent,'.',Sys.Date(),'.tsv'), sep = "\t", row.names=FALSE, quote = FALSE)


In [17]:


##======= Boxplot for two groups compared alteration with none-alteration

library(ggplot2)
library(ggpubr)


## two group with statistic
# dataframe
# altGroup, log2tpm
CompBoxplot2exp <- function(title, dataFrame){   
    
    n_alt_case <- length(unique(dataFrame$CaseID[dataFrame$altGroup=='Deletion']))
    n_other_case <- length(unique(dataFrame$CaseID[dataFrame$altGroup=='WT']))
    
    n_alt_sample <- length(unique(dataFrame$WES[dataFrame$altGroup=='Deletion']))
    n_other_sample <- length(unique(dataFrame$WES[dataFrame$altGroup=='WT']))
    
    label_alt <- paste0("Deletion","\nN=",n_alt_case,"\nn=",n_alt_sample)
    label_other <- paste0("WT","\nN=",n_other_case,"\nn=",n_other_sample)
    
    
    p <- ggplot(dataFrame, aes(x=altGroup, y=log2tpm, color=altGroup)) +
            geom_boxplot(lwd=0.6, outlier.size=0.6) + 
            geom_jitter(position=position_jitter(0.1), size=0.6, alpha = 0.5) +
            theme_classic() +
            scale_color_manual(values = c("WT" = "#B5B5B5", "Deletion" = "#20255D")) + 
            theme(legend.position="None") +
            ggtitle(title) +
            xlab("") +
            ylab("Gene expression\nlog2(TPM+1)") + 
            scale_x_discrete(labels=c("Deletion" = label_alt, "WT" = label_other)) +
            theme(
                plot.title = element_text(size=7, hjust = 0.5),
                axis.text = element_text(size = 6),
                axis.title=element_text(size=6)) + 
            theme(
                axis.ticks = element_line(size = rel(0.5)),
                axis.ticks.length=unit(1.5, "mm"),
                axis.line = element_line(colour = 'black', size = 0.2)
            ) +
            theme(panel.grid.major = element_blank(), 
               panel.grid.minor = element_blank(), 
               panel.background = element_blank(), 
               panel.border = element_blank()
            )

    
    return(p)

}






#---------- f) make visulization (2.plotting)

altEvent <- 'Deletion'

#cancerSet <- unique(d_tar_expDel$CancerType)
cancerSet <- c("BLCA")

for(cancerType in cancerSet){
    
    df_cancer <- d_tar_expDel[d_tar_expDel$CancerType==cancerType,]
    # filter sample size < 10
    if(length(unique(df_cancer$SampleID))<10){ next }

    
    pdf(paste0(outdir_sub,'/',cancerType,'.',altEvent,'_geneExp.',Sys.Date(),'.pdf'), width = 1.4, height = 1.8, useDingbats=FALSE)

    for(i in unique(d_tar_expDel$gene)){
    
        df2 <- df_cancer[df_cancer$gene==i,]
        
        # remove genes 
        # if no any mut then del. (for pdx 1 sample also possible)
        if(nrow(df2[df2$altGroup==altEvent,])==0){ next }
    
        # 2.not significant in gene exp. between two groups
        w_test <- wilcox.test(log2tpm ~ altGroup, data = df2)
        if(w_test$p.value >= 0.05){ next }
        
    
        # output significant
        if (nrow(df2[df2$altGroup==altEvent,])>0 && w_test$p.value < 0.05){
            print(df2[df2$altGroup==altEvent,])
            p_val <- formatC(w_test$p.value, format = "e", digits = 1)
            title <- paste0(i, ' (', cancerType, ')', "\nWilcoxon, p=", p_val)
            
            p1 <- CompBoxplot2exp(title, df2)

            print(p1)
        }
    
    }

    dev.off()

}




                                WES gene                            RNASEQ
2249     BL0382_F1232_M662_v1_2_WES  MET     BL0382_F1232_M662_v1_2_RNASEQ
2251  BL0382_F1232_M662M18_v1_2_WES  MET  BL0382_F1232_M662M18_v1_2_RNASEQ
2253 BL0382_F1232_M664M220_v1_2_WES  MET BL0382_F1232_M664M220_v1_2_RNASEQ
2585    TWDE-WBC-1027-DBLSL10A_2690  MET   tblsl10a_2690.ATATCTCG-ATCTTAGT
2587    TWDE-WBC-1027-DBLSL10B_3043  MET   tblsl10b_3043.GCGCTCTA-GCTCCGAC
2589   TWDE-WBC-1027-DBLSL10C_3141m  MET      TWDE-WBC-1027-TBLSL10C_3141m
2591   TWDE-WBC-1027-DBLSL10D_3280m  MET  tblsl10d_3280m.AACAGGTT-ATACCAAG
           tpm  log2tpm Source CancerGroup CancerType      CaseID
2249 101.30311 6.676706   PDMR     Bladder       BLCA PDMR-BL0382
2251  73.43872 6.217981   PDMR     Bladder       BLCA PDMR-BL0382
2253 119.39957 6.911686   PDMR     Bladder       BLCA PDMR-BL0382
2585 430.78798 8.754179  WashU     Bladder       BLCA     WU-0029
2587 365.02301 8.515791  WashU     Bladder       BLCA     WU-0029
2589

In [18]:

#---------- f) make visulization (1.set plot function)

library(ggplot2)
library(ggpubr)

CompBoxplot2exp_panCancers_per_gene <- function(gene, dataFrame){
    
    df_gene <- dataFrame[dataFrame$gene==gene,]
    
    max.y <- max(df_gene$log2tpm)
    
    p <- ggplot(df_gene, aes(x=CancerType, y=log2tpm, color=altGroup)) +
            geom_boxplot(lwd=0.4, position=position_dodge(1), outlier.size=0.4, pch=16, outlier.alpha=0.5) + 
            geom_point(pch=16, position=position_jitterdodge(0.2), size=1, alpha = 0.5) +
            theme_classic() +
            scale_color_manual(values = c( "WT" = "#7D94AF", "Deletion" = "#20255D")) + 
            ggtitle(gene) +
            theme(plot.title = element_text(size=8,hjust = 0.5),
                    axis.title=element_text(size=7),
                    axis.text.x = element_text(size = 7, angle = 60, hjust = 1),
                    axis.text.y = element_text(size = 7)) + 
            theme(
                axis.ticks = element_line(size = rel(0.5)),
                axis.ticks.length=unit(1.5, "mm"),
                axis.line = element_line(colour = 'black', size = 0.2)) +
            xlab("") +
            ylab("Gene expression\nlog2(TPM+1)") + 
            theme(legend.position=c(0.9, 0.9),
                    legend.title=element_blank(),
                    legend.key.size = unit(0.5, "cm"),
                    legend.text=element_text(size=7)
            ) + 
            ylim(0,15)
    
    return(p)

}




#---------- f) make visulization (2.plotting)

pdf(paste0(outdir_sub,'/','2c.all.target.geneExp_with_deltion.',Sys.Date(),'.pdf'), width = 7.5, height = 2.5, useDingbats=FALSE)

for(i in unique(d_tar_expDel$gene)){
    
    df2 <- d_tar_expDel[d_tar_expDel$gene==i,]
    
    if(nrow(df2[df2$altGroup=='Deletion',])==0){
        next
    }
    
    p <- CompBoxplot2exp_panCancers_per_gene(i, df2)
    print(p)

}

dev.off()



pdf 
  2

# 3. CNV and gene expression (c. Loss)

In [19]:

# outdir
outdir_sub <- "../3.cnv_loss_with_geneExp"
if (! dir.exists(outdir_sub)) { dir.create(outdir_sub) }


#---------- a) extract drug target
#loss_tarGeneSet <- d_drugSet[d_drugSet$Event==altEvent,]$Gene
loss_tarGeneSet <- d_drugSet[d_drugSet$Event %in% c('Loss','Deletion'),]$Gene


#---------- b) extract mutation
d_cnv <- fread(f_cnv, data.table=F)

# sample gene segment_log2 segment_cn gene_log2  gene_cn
d_cnv_v2 <- d_cnv[, c('sample', 'gene', 'segment_log2', 'segment_cn')]


d_loss_tar <- d_cnv_v2 %>%
                filter(sample %in% d_pdx_info$WES) %>%
                filter(gene %in% loss_tarGeneSet) %>%
                filter(segment_log2 < -0.4)
                #filter(segment_cn ==1)

rownames(d_loss_tar) <- NULL


# output text file for checking
#write.table(d_loss_tar, paste0(outdir_sub,'/3a.all.target.loss.',Sys.Date(),'.tsv'), sep = "\t", row.names=FALSE, quote = FALSE)
write.table(d_loss_tar, paste0(outdir_sub,'/3a.all.target.loss_del.',Sys.Date(),'.tsv'), sep = "\t", row.names=FALSE, quote = FALSE)





#---------- c) extract gene expression

# format gene exp data frame as --> sample gene log2tpm
d_geneExp_tar <- subset(d_geneExp, Name %in% loss_tarGeneSet)
d_geneExp_tar <- reshape2::melt(d_geneExp_tar, id.vars=c("Name"))
colnames(d_geneExp_tar) <- c('gene', 'RNASEQ', 'tpm')
d_geneExp_tar <- d_geneExp_tar[, c('RNASEQ', 'gene', 'tpm')]
# RNASEQ gene tpm

# tpm --> log2(tpm+1)
d_geneExp_tar$log2tpm <- log2(d_geneExp_tar$tpm + 1)

d_geneExp_tar <- d_geneExp_tar %>% filter(RNASEQ %in% d_pdx_info$RNASEQ)
rownames(d_geneExp_tar) <- NULL
# RNASEQ gene tpm log2tpm


#---------- d) merge formatted gene exp with somatic mut datafrme

#NOTE- must cnv data merge to gene exp rather than gene exp merge to cnv

#1. merge data info
d_tar_expLoss <- merge(d_geneExp_tar, d_pdx_info, on='RNASEQ', all.x=TRUE, all.y=FALSE)
#2. merge cnv
d_tar_expLoss <- merge(d_tar_expLoss, d_loss_tar, by.x=c('WES', 'gene'), by.y=c('sample', 'gene'), all.x=TRUE, all.y=FALSE)

# 3. add PDX model ID
d_tar_expLoss$PDXmodelID <- d_sampleInfo$ModelID_proper[match(d_tar_expLoss$WES, d_sampleInfo$Analysis_ID)]

#---------- e) add flag with Mut & Non-mut
d_tar_expLoss$altGroup <- 'WT'
d_tar_expLoss$altGroup[d_tar_expLoss$segment_cn != 'NA'] <- 'Loss'


# output text file for checking
#write.table(d_tar_expLoss, paste0(outdir_sub,'/3b.all.target.geneExp_with_loss.',Sys.Date(),'.tsv'), sep = "\t", row.names=FALSE, quote = FALSE)
write.table(d_tar_expLoss, paste0(outdir_sub,'/3b.all.target.geneExp_with_loss_del.',Sys.Date(),'.tsv'), sep = "\t", row.names=FALSE, quote = FALSE)



head(d_tar_expLoss)




WES,gene,RNASEQ,tpm,log2tpm,Source,CancerGroup,CancerType,CaseID,SampleID,Group,Passage,segment_log2,segment_cn,PDXmodelID,altGroup
10960X12_HCI-009_PDXtumor_P3,MET,10962X12_HCI009_PDXtumor_P3,11.29700740,3.62023536,HCI,Breast,BRCA,HCI-HCI009,HCI-HCI009.P3,PDX,P3,NA,NA,HCI009,WT
10960X12_HCI-009_PDXtumor_P3,MLH1,10962X12_HCI009_PDXtumor_P3,8.60108803,3.26319791,HCI,Breast,BRCA,HCI-HCI009,HCI-HCI009.P3,PDX,P3,-0.636702,1,HCI009,Loss
10960X12_HCI-009_PDXtumor_P3,MSH2,10962X12_HCI009_PDXtumor_P3,11.40290670,3.63260636,HCI,Breast,BRCA,HCI-HCI009,HCI-HCI009.P3,PDX,P3,NA,NA,HCI009,WT
10960X12_HCI-009_PDXtumor_P3,MSH6,10962X12_HCI009_PDXtumor_P3,20.69513210,4.43929947,HCI,Breast,BRCA,HCI-HCI009,HCI-HCI009.P3,PDX,P3,NA,NA,HCI009,WT
10960X12_HCI-009_PDXtumor_P3,PMS2,10962X12_HCI009_PDXtumor_P3,8.99987900,3.32191064,HCI,Breast,BRCA,HCI-HCI009,HCI-HCI009.P3,PDX,P3,NA,NA,HCI009,WT
10960X12_HCI-009_PDXtumor_P3,PTEN,10962X12_HCI009_PDXtumor_P3,0.05750863,0.08066944,HCI,Breast,BRCA,HCI-HCI009,HCI-HCI009.P3,PDX,P3,-6.803390,0,HCI009,Loss


In [20]:
#--------- (Important checking) make text file for record siginficant deletion
alterationEvent <- 'Loss'
d_rec <- func_rec_sigDiffExp_in_alt(d_tar_expLoss, alterationEvent)
write.table(d_rec, paste0(outdir_sub,'/3.p-val.target.geneExp_with_',alterationEvent,'.',Sys.Date(),'.tsv'), sep = "\t", row.names=FALSE, quote = FALSE)



Warning message in wilcox.test.default(x = c(4.0909638710484, 4.41350846046682, :
“cannot compute exact p-value with ties”Warning message in wilcox.test.default(x = c(7.09538218326582, 2.60070459010433, :
“cannot compute exact p-value with ties”Warning message in wilcox.test.default(x = c(0.11535379202118, 0.0998152221225018, :
“cannot compute exact p-value with ties”Warning message in wilcox.test.default(x = c(0, 0, 0, 6.39309239868405, 5.74573308057332, :
“cannot compute exact p-value with ties”

In [21]:

#---------- f) make visulization (1.set plot function)

library(ggplot2)
library(ggpubr)

CompBoxplot2exp_panCancers_per_gene <- function(gene, dataFrame){
    
    df_gene <- dataFrame[dataFrame$gene==gene,]
    
    max.y <- max(df_gene$log2tpm)
    
    p <- ggplot(df_gene, aes(x=CancerType, y=log2tpm, color=altGroup)) +
            geom_boxplot(lwd=0.4, position=position_dodge(1), outlier.size=0.4, pch=16, outlier.alpha=0.5) + 
            geom_point(pch=16, position=position_jitterdodge(0.2), size=1, alpha = 0.5) +
            theme_classic() +
            scale_color_manual(values = c("WT" = "#7D94AF", "Loss" = "#98509F")) + 
            ggtitle(gene) +
            theme(plot.title = element_text(size=8,hjust = 0.5),
                    axis.title=element_text(size=7),
                    axis.text.x = element_text(size = 7, angle = 60, hjust = 1),
                    axis.text.y = element_text(size = 7)) + 
            theme(
                axis.ticks = element_line(size = rel(0.5)),
                axis.ticks.length=unit(1.5, "mm"),
                axis.line = element_line(colour = 'black', size = 0.2)) +
            xlab("") +
            ylab("Gene expression\nlog2(TPM+1)") + 
            theme(legend.position=c(0.9, 0.9),
                    legend.title=element_blank(),
                    legend.key.size = unit(0.5, "cm"),
                    legend.text=element_text(size=7)
            ) + 
            theme(panel.grid.major = element_blank(), 
               panel.grid.minor = element_blank(), 
               panel.background = element_blank(), 
               panel.border = element_blank()
            ) +
            ylim(0,15)
    
    return(p)

}




#---------- f) make visulization (2.plotting)

#pdf(paste0(outdir_sub,'/','3c.all.target.geneExp_with_loss.',Sys.Date(),'.pdf'), width = 7.5, height = 2.5, useDingbats=FALSE)
pdf(paste0(outdir_sub,'/','3c.all.target.geneExp_with_loss_del.',Sys.Date(),'.pdf'), width = 7.5, height = 2.5, useDingbats=FALSE)


for(i in unique(d_tar_expLoss$gene)){
    
    df2 <- d_tar_expLoss[d_tar_expLoss$gene==i,]
    
    if(nrow(df2[df2$altGroup=='Loss',])==0){
        next
    }
    
    p <- CompBoxplot2exp_panCancers_per_gene(i, df2)
    print(p)

}

dev.off()



pdf 
  2

In [22]:

##=================== Make loss vs no loss boxplot for specific gene and cancerType


# outdir
outdir_sub <- "../3.cnv_loss_with_geneExp"
if (! dir.exists(outdir_sub)) { dir.create(outdir_sub) }



#---------- g) make visulization (1.set plot function)

library(ggplot2)
library(ggpubr)


## two group with statistic
CompBoxplot2exp <- function(title, dataFrame){   
    
    #n_alt_case <- length(unique(dataFrame$CaseID[dataFrame$altGroup=='Loss']))
    #n_other_case <- length(unique(dataFrame$CaseID[dataFrame$altGroup=='WT']))
    n_alt_case <- length(unique(dataFrame$PDXmodelID[dataFrame$altGroup=='Loss']))
    n_other_case <- length(unique(dataFrame$PDXmodelID[dataFrame$altGroup=='WT']))
    
    n_alt_sample <- length(unique(dataFrame$WES[dataFrame$altGroup=='Loss']))
    n_other_sample <- length(unique(dataFrame$WES[dataFrame$altGroup=='WT']))
    
    label_alt <- paste0("Loss","\nN=",n_alt_case,"\nn=",n_alt_sample)
    label_other <- paste0("WT","\nN=",n_other_case,"\nn=",n_other_sample)
    
    
    p <- ggplot(dataFrame, aes(x=altGroup, y=log2tpm, color=altGroup)) +
            geom_boxplot(lwd=0.6, outlier.size=0.6) + 
            geom_jitter(position=position_jitter(0.1), size=0.6, alpha = 0.5) +
            theme_classic() +
            scale_color_manual(values = c("WT" = "#B5B5B5", "Loss" = "#98509F")) + 
            theme(legend.position="None") +
            ggtitle(title) +
            xlab("") +
            ylab("Gene expression\nlog2(TPM+1)") + 
            scale_x_discrete(
                limit = c("WT", "Loss"),
                labels=c("Loss" = label_alt, "WT" = label_other)) +
            theme(
                plot.title = element_text(size=7, hjust = 0.5),
                axis.text = element_text(size = 6),
                axis.title=element_text(size=6)) + 
            theme(
                axis.ticks = element_line(size = rel(0.5)),
                axis.ticks.length=unit(1.5, "mm"),
                axis.line = element_line(colour = 'black', size = 0.2)
            ) +
            theme(panel.grid.major = element_blank(), 
               panel.grid.minor = element_blank(), 
               panel.background = element_blank(), 
               panel.border = element_blank()
            )

    
    return(p)

}



#---------- g) make visulization (2.plotting)

cancerTypeSet <- c('SARC', 'SKCM', 'PAAD', 'COAD')

for(cancerType in cancerTypeSet){
    
    df_cancer <- d_tar_expLoss[d_tar_expLoss$CancerType==cancerType,]
    # filter sample size < 10
    if(length(unique(df_cancer$SampleID))<10){ next }
    
    
    pdf(paste0(outdir_sub,'/',cancerType,'.loss_geneExp.',Sys.Date(),'.pdf'), width = 1.4, height = 1.8, useDingbats=FALSE)

    for(i in unique(d_tar_expLoss$gene)){
    
        df2 <- df_cancer[df_cancer$gene==i,]
        
        # remove genes 
        # if no any mut then del. (for pdx 1 sample also possible)
        if(nrow(df2[df2$altGroup=='Loss',])==0){ next }
    
        # 2.not significant in gene exp. between two groups
        w_test <- wilcox.test(log2tpm ~ altGroup, data = df2)
        if(w_test$p.value >= 0.01){ next }
    
    
        # output significant
        if (nrow(df2[df2$altGroup=='Loss',])>0 && w_test$p.value < 0.01){
            
            p_val <- formatC(w_test$p.value, format = "e", digits = 1)
            title <- paste0(i, ' (', cancerType, ')', "\nWilcoxon, p=", p_val)
            
            p1 <- CompBoxplot2exp(title, df2)

            print(p1)
        }
    
    }

    dev.off()

}

head(d_tar_expLoss)


WES,gene,RNASEQ,tpm,log2tpm,Source,CancerGroup,CancerType,CaseID,SampleID,Group,Passage,segment_log2,segment_cn,PDXmodelID,altGroup
10960X12_HCI-009_PDXtumor_P3,MET,10962X12_HCI009_PDXtumor_P3,11.29700740,3.62023536,HCI,Breast,BRCA,HCI-HCI009,HCI-HCI009.P3,PDX,P3,NA,NA,HCI009,WT
10960X12_HCI-009_PDXtumor_P3,MLH1,10962X12_HCI009_PDXtumor_P3,8.60108803,3.26319791,HCI,Breast,BRCA,HCI-HCI009,HCI-HCI009.P3,PDX,P3,-0.636702,1,HCI009,Loss
10960X12_HCI-009_PDXtumor_P3,MSH2,10962X12_HCI009_PDXtumor_P3,11.40290670,3.63260636,HCI,Breast,BRCA,HCI-HCI009,HCI-HCI009.P3,PDX,P3,NA,NA,HCI009,WT
10960X12_HCI-009_PDXtumor_P3,MSH6,10962X12_HCI009_PDXtumor_P3,20.69513210,4.43929947,HCI,Breast,BRCA,HCI-HCI009,HCI-HCI009.P3,PDX,P3,NA,NA,HCI009,WT
10960X12_HCI-009_PDXtumor_P3,PMS2,10962X12_HCI009_PDXtumor_P3,8.99987900,3.32191064,HCI,Breast,BRCA,HCI-HCI009,HCI-HCI009.P3,PDX,P3,NA,NA,HCI009,WT
10960X12_HCI-009_PDXtumor_P3,PTEN,10962X12_HCI009_PDXtumor_P3,0.05750863,0.08066944,HCI,Breast,BRCA,HCI-HCI009,HCI-HCI009.P3,PDX,P3,-6.803390,0,HCI009,Loss
